<a href="https://colab.research.google.com/github/PNG-bot/NLP_WiSe_24_25/blob/MUFF_NLP_24_25/eigenes_projekt/eigenes_projekt_zeitschriften_einlesen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eigenes Projekt - Zeitschriften aus DZA einlesen